## Data loading

In [ ]:
import keras
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Reshape, UpSampling2D, Flatten, Dense, Lambda, BatchNormalization
from keras.metrics import BinaryAccuracy
from keras.models import Model
import keras.backend as K
import tensorflow as tf
from keras.losses import mse
from keras.callbacks import EarlyStopping

from scipy import ndimage
import cv2
import os
import numpy as np
from imutils import build_montages

from google.colab.patches import cv2_imshow
from google.colab import drive

from tqdm import tqdm
import shutil

In [ ]:
!mkdir /content/images
!mkdir /content/images/train
!mkdir /content/images/test
!mkdir /content/labels
!mkdir /content/labels/train
!mkdir /content/labels/test
!mkdir /content/rotations
!mkdir /content/rotations/train
!mkdir /content/rotations/test

In [ ]:
drive.mount('/content/drive')

In [ ]:
!unzip -q '/content/drive/My Drive/NATO/train2904.zip' -d '/content/images/train'
!unzip -q '/content/drive/My Drive/NATO/train_l2904.zip' -d '/content/labels/train'
!unzip -q '/content/drive/My Drive/NATO/train_r2904.zip' -d '/content/rotations/train'
!unzip -q '/content/drive/My Drive/NATO/test2904.zip' -d '/content/images/test'
!unzip -q '/content/drive/My Drive/NATO/test_l2904.zip' -d '/content/labels/test'
!unzip -q '/content/drive/My Drive/NATO/test_r2904.zip' -d '/content/rotations/test'

In [ ]:
Dict = {0: 'advance_to_contact', 2: 'attack', 9: 'counterattack', 18: 'main_attack'}

In [ ]:
Dict_images = {key: [] for key in Dict.values()}
Dict_images_test = {key: [] for key in Dict.values()}

In [ ]:
images = '/content/images/train'
labels = '/content/labels/train'
rotations = '/content/rotations/train'

file_list = os.listdir(images)

for name in tqdm(file_list):
    tmp_list_class = []
    image = cv2.imread(images+"/"+name, cv2.IMREAD_GRAYSCALE)
    imageW = image.shape[1]
    imageH = image.shape[0]
    with open(labels+"/"+str(name).split(".")[0]+".txt") as f:
        lines = [line.rstrip('\n') for line in f]
    with open(rotations+"/"+str(name).split(".")[0]+".txt") as f:
        rots = [line.rstrip('\n') for line in f]
        for nr, line in enumerate(lines):
            x = int((float(line.split(" ")[1])-(float(line.split(" ")[3])/2)) * imageW)
            y = int((float(line.split(" ")[2])-(float(line.split(" ")[4])/2)) * imageH)
            w = int(float(line.split(" ")[3]) * imageW + 5)
            h = int(float(line.split(" ")[4]) * imageH + 5)

            crop_image = image[y:y+h, x:x+w]
            tmp_list_class.append(int(line.split(" ")[0]))
            crop_image = ndimage.rotate(crop_image, -int(rots[nr]), mode='constant', cval=255) ### rotation aspekt
            dim = (80, 80)
            resized_img = cv2.resize(crop_image, dim, interpolation = cv2.INTER_AREA)
            binImg = img = np.where(resized_img <= 150, 0, 255)
            try:
                Dict_images[Dict[int(line.split(" ")[0])]].append(binImg)
            except KeyError:
                pass

In [ ]:
images = '/content/images/test'
labels = '/content/labels/test'
rotations = '/content/rotations/test'

file_list = os.listdir(images)

for name in tqdm(file_list):
    tmp_list_class = []
    image = cv2.imread(images+"/"+name, cv2.IMREAD_GRAYSCALE)
    with open(labels+"/"+str(name).split(".")[0]+".txt") as f:
        lines = [line.rstrip('\n') for line in f]
    with open(rotations+"/"+str(name).split(".")[0]+".txt") as f:
        rots = [line.rstrip('\n') for line in f]
        for nr, line in enumerate(lines):
            x = int((float(line.split(" ")[1])-(float(line.split(" ")[3])/2)) * 770)
            y = int((float(line.split(" ")[2])-(float(line.split(" ")[4])/2)) * 576)
            w = int(float(line.split(" ")[3]) * 770 + 5)
            h = int(float(line.split(" ")[4]) * 576 + 5)

            crop_image = image[y:y+h, x:x+w]
            tmp_list_class.append(int(line.split(" ")[0]))
            crop_image = ndimage.rotate(crop_image, -int(rots[nr]), mode='constant', cval=255) ### rotation aspekt
            dim = (80, 80)
            resized_img = cv2.resize(crop_image, dim, interpolation = cv2.INTER_AREA)
            binImg = img = np.where(resized_img <= 150, 0, 255)
            try:
                Dict_images_test[Dict[int(line.split(" ")[0])]].append(binImg)
            except KeyError:
                pass

## Model

In [ ]:
def Sampling(inputs):
    mean, log_var = inputs
    return K.random_normal(tf.shape(mean)) * K.exp(log_var / 2) + mean

def model_version12():
    # autoencoder model
    beta_value = 50
    # encoder
    input = Input(shape=(80, 80, 1))
    x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(input)
    x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
    x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
    x = Conv2D(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    codings_mean = keras.layers.Dense(10, name="encoder_mean")(x)
    codings_log_var = keras.layers.Dense(10, name="encoder_log_var")(x)
    codings = Lambda(Sampling, name="encoder_output")([codings_mean, codings_log_var])
    variational_encoder = keras.models.Model(inputs=[input], outputs=[codings_mean, codings_log_var, codings], name="encoder")

    #decoder
    decoder_inputs = keras.layers.Input(shape=(10))
    z = Dense(256, activation='relu')(decoder_inputs)
    z = Dense(256, activation='relu')(z)
    z = Dense(5*5*32, activation='relu')(z)
    z = Reshape((5, 5, 32))(z)
    z = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(z)
    z = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(z)
    z = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(z)
    z = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(z)
    output = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(z)
    variational_decoder = keras.models.Model(inputs=[decoder_inputs], outputs=[output], name="decoder")

    _, _, codings = variational_encoder(input)
    reconstructions = variational_decoder(codings)
    variational_ae = keras.models.Model(inputs=[input], outputs=[reconstructions], name="autoencoder")

    reconstruction_loss_factor = 1000
    reconstruction_loss = mse(K.flatten(input), K.flatten(reconstructions))
    reconstruction_loss *= 80 * 80 * 1
    kl_loss = -0.5 * beta_value * K.sum(1 + codings_log_var - K.square(codings_mean) - K.exp(codings_log_var), axis=1)
    vae_loss = K.mean(reconstruction_loss_factor * reconstruction_loss + kl_loss)
    variational_ae.add_loss(vae_loss)

    variational_ae.add_metric(kl_loss, name="kl_loss")
    variational_ae.add_metric(reconstruction_loss, name="reconstruction_loss")

    return variational_ae

## Training

In [ ]:
for i in Dict_images.keys():
    print(str(i))
    x_train = np.array(Dict_images[str(i)])
    x_train = x_train.astype('float32') / 255.
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)

    x_test = np.array(Dict_images_test[str(i)])
    x_test = x_test.astype('float32') / 255.
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

    print(x_train.shape, x_test.shape)

    # earlyStopping = EarlyStopping(monitor='val_reconstruction_loss', start_from_epoch=50, patience=20, restore_best_weights=True, verbose=1, mode="min")

    model = model_version12()

    # Compile the model
    model.compile(optimizer='adam')

    # Fit the model
    model.fit(x_train, x_train, batch_size = 128, epochs = 200, validation_data=(x_test, x_test))

    model.save(str(i)+"_autoencoder_model12_b50_10.h5")

    colab_link = '/content/'+str(i)+'_autoencoder_model12_b50_10.h5'
    gdrive_link = "/content/drive/My Drive/models_trajectory_final/"
    shutil.copy(colab_link, gdrive_link)

In [ ]:
for i in Dict_images_test.keys():
    str(i)
    model = model_version12()
    model.load_weights('/content/drive/MyDrive/models_trajectory_final/'+str(i)+'_autoencoder_model12_b3_10.h5')

    images = np.array(Dict_images_test[str(i)])
    images = images.astype('float32') / 255.
    images = images.reshape(images.shape[0], images.shape[1], images.shape[2], 1)

    decoded_images = model.predict(images)

    number_of_images = list(range(0, images.shape[0]))
    random_images = np.random.choice(number_of_images, size=10, replace=False)

    examples = []
    for j in random_images:
        original = images[j] * 255
        decoded_img = decoded_images[j] * 255
        comparison = np.hstack([original, decoded_img])
        examples.append(comparison)

    comparison_images = np.vstack(examples)
    cv2_imshow(comparison_images)